In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

import h5py

# E906 Messy MC Data

Let $\beta_{0}$ and $\beta_{1}$ define two distributions, $p(x|\beta_{0})$ and $p(x|\beta_{1})$ respectively. Then the likelihood ratio is defined by;
$$
\mathcal{L}(x| \beta_{0}, \beta_{1}) =  \frac{p(x|\beta_{0})}{p(x|\beta_{1})}
$$

A classifier function $f$, designed to distinguish samples drawn from $p(x|\beta_{0})$ and $p(x|\beta_{1})$, can be used to approximate likelihood ratios;

$$
\mathcal{L}(x| \beta_{0}, \beta_{1}) =  \frac{f(x, \beta_{0}, \beta_{1})}{1 - f(x, \beta_{0}, \beta_{1})}
$$

Consider the cross-section of the Drell-Yan (DY) angular distribution;

$$
\frac{d\sigma}{d\Omega} \propto 1 + \lambda \cos^{2}\theta + \mu\sin2\theta\cos\phi + \frac{\nu}{2}\sin^{2}\theta\cos2\phi
$$

Our goal is to extract Drell-Yan (DY) angular coefficients, $\lambda$, $\mu$, and $\nu$, using the likelihood ratio method. We aim to train a classifier (neural network) capable of classifying two samples. We sample $\lambda$, $\mu$, and $\nu$ from uniform ranges: $\lambda$ in $[-1, 1]$, $\mu$ and $\nu$ in $[-0.5, 0.5]$. We extract the bin center and bin content of $\phi$ vs. $\cos\theta$ histograms as inputs and weights in the classifier and loss function. The two classes are defined as follows:

```
H0: (lambda=0, mu=0, nu=0, phi, costheta) with beta = (lambda, mu, nu) --> label = 0
H1: (lambda, mu, nu, phi, costheta) with beta = (lambda, mu, nu) --> label = 1
```

Note that the class with label 0 has mismatching $\beta$ values.

In [2]:
# save outputs
save = h5py.File("fit.hdf5", "w")

In [ ]:
# model used for likelihood learning
class NetClassifier(nn.Module):
    def __init__(self, input_dim: int=5, output_dim: int=1, hidden_dim: int=32):
        super(NetClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim, bias=True)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.fc5 = nn.Linear(hidden_dim, output_dim, bias=True)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))
        return x